* Remove all drugs unseen in test_16.pkl
* Generate feature matrices

** Run this script on a machine with more than 16GB memory **

Input:
* train_16.pkl, train_16_ground_truth.pkl
* test_16.pkl
* train_15.pkl, train_15_ground_truth.pkl
* test_15.pkl, test_15_ground_truth.pkl

Output:
* feature_train_16.pkl
* feature_test_16.pkl
* feature_train_15.pkl
* feature_test_15.pkl

In [1]:
from IPython.display import display

import time
import numpy as np
import pandas as pd
from scipy.sparse import dok_matrix, csr_matrix

import pickle

def load_data(filename):
    with open(filename, 'rb') as fin:
        return pickle.load(fin)

def save_data(obj, filename):
    with open(filename, 'wb+') as fout:
        pickle.dump(obj, fout)
        
import os
root_path = '/home/yuanl4/Datathon/data/'

files = [('test_16.pkl',),
         ('test_15.pkl', 'test_15_ground_truth.pkl'),
         ('train_16.pkl', 'train_16_ground_truth.pkl'),
         ('train_15.pkl', 'train_15_ground_truth.pkl')]

def check_file(file):
    assert os.path.isfile(file), "File doesn't exist: %s" % file

for pair in files:
    for file in pair:
        check_file(root_path + file)

In [2]:
drug_lookup_full = pd.read_csv('drug_with_illness_lookup_CN.csv', sep='\t', encoding='utf-16', index_col='DID')

In [3]:
transactions = load_data(root_path + 'test_16.pkl')
assert (np.in1d(transactions[:, 1], drug_lookup_full.index.values)).all(), 'some drugs are not in drug_lookup'

In [4]:
unique_drugs_in_test = np.unique(transactions[:, 1])
code_seen_in_test = np.unique(drug_lookup_full.loc[unique_drugs_in_test, ['L1C', 'L2C', 'L3C', 'L4C']].values)
code_seen_in_test = np.concatenate([['#'], code_seen_in_test])

In [5]:
encoded_code_lookup = pd.DataFrame(
    data=np.searchsorted(code_seen_in_test, 
                         drug_lookup_full.loc[unique_drugs_in_test, ['L1C', 'L2C', 'L3C', 'L4C']].values),
    columns=['L1C', 'L2C', 'L3C', 'L4C'],
    index=unique_drugs_in_test
)

In [6]:
code_features = ['Beg', 'End', 'Dur', 'GMed', 'GMax', 'y-3', 'y-2', 'y-1', 'h1', 'q3', 'q4',
                 'm06', 'm12', 'm24', 'm36', 'm48', 'Bdif', 'Edif', 'NPD1', 'NPD2']

In [7]:
feature_names = [code + '_' + cf for code in code_seen_in_test for cf in code_features]
print('feature_names', len(feature_names), feature_names[:21])

feature_names 14320 ['#_Beg', '#_End', '#_Dur', '#_GMed', '#_GMax', '#_y-3', '#_y-2', '#_y-1', '#_h1', '#_q3', '#_q4', '#_m06', '#_m12', '#_m24', '#_m36', '#_m48', '#_Bdif', '#_Edif', '#_NPD1', '#_NPD2', 'A_Beg']


functions

In [8]:
def group_min_max_range_histogram_gap_med_max(groups, values, bins):
    unique_groups = np.unique(groups)
    output = np.zeros((unique_groups.shape[0], bins.shape[0]+5), dtype=np.int16)
    # group_ID, min, max, range, gmed, gmax, histogram
    # 1,        1,   1,   1,     1,    1,    len(bins)-1
    # 0,        1,   2,   3,     4,    5,    6:
    for i, g in enumerate(unique_groups):
        values_selected = values[groups==g]
        output[i, :3] = [g, values_selected.min(), values_selected.max()] # DID, min, max
        output[i, 6:] = np.histogram(values_selected, bins)[0]            # histogram
        gaps = np.ediff1d(values_selected)       # gaps
        gaps = gaps[gaps!=0]                     # remove 0s
        if gaps.shape[0] != 0:
            output[i, 4] = np.median(gaps)       # gap median
            output[i, 5] = gaps.max()            # gap max
    output[:, 3] = output[:, 2] - output[:, 1]
    return output

def generate_feature_matrix_for_one_instance(matrix, encoded_ref_date, bins):
    if matrix.shape[0] == 0:
        print('error')
        return matrix

    atc_code_groups = encoded_code_lookup.values[matrix[:, 1]]
    atc_features = [group_min_max_range_histogram_gap_med_max(np.zeros(matrix.shape[0]), matrix[:, 2], bins)]
    for i in range(4):
        atc_features.append(group_min_max_range_histogram_gap_med_max(atc_code_groups[:, i], matrix[:, 2], bins))
    feature_matrix = np.vstack(atc_features)
    
    feature_matrix[:, 2] -= encoded_ref_date # End -> ref date
    
    feature_matrix = np.hstack([feature_matrix,                                  # DID, Beg, End, Dur, GMed, GMax, hist
                                feature_matrix[:, :5:-1].cumsum(axis=1)[:, 1:],  # cumulative hist
                                feature_matrix[:, [1]]-feature_matrix[0, 1]  +1, # Begd：Beg-#_Beg+1
                                feature_matrix[0, 2]  -feature_matrix[:, [2]]+1, # Endd: #_End-End+1
                                feature_matrix[:, 4:6]+feature_matrix[:, [2]]])  # NS1: End+GMed, NS2: End+GMax
    return feature_matrix

In [9]:
def generate_feature_matrix(patients, encoded_ref_date, bins):
    num_instances = len(patients)
    print('total number of instances:', str(num_instances/1000)+'k')
    
    sparse_feature_matrix = dok_matrix((num_instances, len(code_seen_in_test)*len(code_features)), dtype=np.int16)
    for i in range(num_instances):
        if i%1000 == 0:
            print(i//1000, end=' ')
            
        if patients[i].shape[0] == 0:
            print('\n#%d has no record#'%i)
            continue
            
        feature_matrix = generate_feature_matrix_for_one_instance(patients[i], encoded_ref_date, bins)
        for row, col in zip(*feature_matrix.nonzero()):
            if col == 0:
                continue
            sparse_feature_matrix[i, feature_matrix[row, 0]*len(code_features)+(col-1)] = feature_matrix[row, col]
        
    print('finished.')
    return sparse_feature_matrix

Make feature matrix

In [10]:
def filter_out_unseen_drugs_encode_drugs_and_get_patients_from_transactions(transactions):
    # remove drugs not seen in test
    print(transactions.shape, ' -> ', end='')
    transactions = transactions[np.in1d(transactions[:, 1], unique_drugs_in_test)]
    print(transactions.shape)
    
    # encode drugs according to unique_drugs_seen_in_test
    transactions[:, 1] = np.searchsorted(unique_drugs_in_test, transactions[:, 1])
    
    return np.asanyarray(np.split(transactions, np.bincount(transactions[:, 0]).cumsum()[:-1]))

In [11]:
def encode_dates(dates):
    year2000 = np.datetime64('2000-01-01')
    return (dates - year2000).astype('timedelta64[D]').astype(int)

year2016 = np.datetime64('2016-01-01')
year2015 = np.datetime64('2016-01-01')
year2000 = np.datetime64('2000-01-01')

bins_2016 = encode_dates(np.asarray(['2012-01-01', '2013-01-01', '2014-01-01',
                                     '2015-01-01', '2015-07-01', '2015-10-01', '2016-01-01'], dtype='datetime64[D]'))
bins_2015 = encode_dates(np.asarray(['2011-01-01', '2012-01-01', '2013-01-01',
                                     '2014-01-01', '2014-07-01', '2014-10-01', '2015-01-01'], dtype='datetime64[D]'))

In [12]:
patients = filter_out_unseen_drugs_encode_drugs_and_get_patients_from_transactions(load_data(root_path + 'train_16.pkl'))
df = pd.DataFrame(generate_feature_matrix_for_one_instance(patients[7], encode_dates(year2016), bins_2016), 
                  columns=np.concatenate([['ATC'], code_features]))
df.set_index('ATC', inplace=True)
df.index = code_seen_in_test[df.index.values]
display(df)

(27622342, 3)  -> (27620133, 3)


,Beg,End,Dur,GMed,GMax,y-3,y-2,y-1,h1,q3,q4,m06,m12,m24,m36,m48,Bdif,Edif,NPD1,NPD2
#,4362,-5,1477,28,329,2,3,8,6,7,2,9,15,23,26,28,1,1,23,324
A,4362,-173,1309,290,329,1,1,1,2,1,0,1,3,4,5,6,1,169,117,156
C,5076,-5,763,28,119,0,1,7,4,4,2,6,10,17,18,18,715,1,23,114
N,4362,-131,1351,325,665,1,1,0,0,2,0,2,2,2,3,4,1,127,194,534
A02,4362,-173,1309,290,329,1,1,1,2,1,0,1,3,4,5,6,1,169,117,156
C07,5657,-5,182,28,126,0,0,0,1,2,1,3,4,4,4,4,1296,1,23,121
C09,5076,-61,707,28,119,0,1,7,3,2,1,3,6,13,14,14,715,57,-33,58
N02,4362,-131,1351,325,665,1,1,0,0,2,0,2,2,2,3,4,1,127,194,534
A02B,4362,-173,1309,290,329,1,1,1,2,1,0,1,3,4,5,6,1,169,117,156
C07A,5657,-5,182,28,126,0,0,0,1,2,1,3,4,4,4,4,1296,1,23,121


In [13]:
def generate(encoded_ref_date, bins, transactions_file, ground_truth_file, output_file):
    print(time.ctime(), 'loading data...', )
    patients = filter_out_unseen_drugs_encode_drugs_and_get_patients_from_transactions(load_data(root_path + transactions_file))

    print(time.ctime(), 'generating features...', )
    X = generate_feature_matrix(patients, encoded_ref_date, bins).tocsr()

    print(time.ctime(), 'finished.')
    if ground_truth_file is not None:
        y = load_data(root_path + ground_truth_file)
        save_data({'X':X, 'y':y, 'feature_names':feature_names}, root_path + output_file)
    else:
        save_data({'X':X, 'feature_names':feature_names}, root_path + output_file)

    print(time.ctime(), 'saved.')

In [14]:
# generate(encode_dates(year2016), bins_2016, 'train_16.pkl', 'train_16_ground_truth.pkl', 'feature_train_16.pkl')
generate(encode_dates(year2016), bins_2016, 'test_16.pkl', None, 'feature_test_16.pkl')
# generate(encode_dates(year2015), bins_2015, 'train_15.pkl', 'train_15_ground_truth.pkl', 'feature_train_15.pkl')
# generate(encode_dates(year2015), bins_2015, 'test_15.pkl', 'test_15_ground_truth.pkl', 'feature_test_15.pkl')

Sun Jun  4 23:42:11 2017 loading data...
(27600052, 3)  -> (27600052, 3)
Sun Jun  4 23:42:17 2017 generating features...
total number of instances: 279.152k
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
#25444 has no record#
26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 
#77930 has no record#
78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 
#171767 has no record#
172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220